# 3. Feladatsor: megoldások
*(Normálegyenlet, iteratív megoldók, gradiens-ereszkedés)*

### P1. Feladat
Írjunk programot, amely egy $A$ SZPD mátrix esetén egy ábrán ábrázolja az $I-\omega A$ mátrix sajátértékeinek abszolútértékét az $\omega$ függvényeként. Bemeneti paraméterek lehetnek a mátrix sajátértékei.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def vplot(*lambdas: float, omega_min: float = 0., omega_max: float = 1.):
    assert len(lambdas) >= 1
    assert omega_max > omega_min
    
    ax = plt.axes()
    
    omegas = np.linspace(omega_min, omega_max, int(101*abs(omega_max - omega_min)))
    for lam in lambdas:
        ax.plot(omegas,  abs(1 - omegas*lam), label=fr"$\lambda$={lam}")

    ax.legend()
    ax.set_xlabel(r"$\omega$")
    ax.set_ylabel(r"$|1-\omega\lambda|$")

vplot(1,2,3,4)

### P2. Feladat
Írjunk általános függvényt a fenti, $A = M-N$ felbontással adódó iterációkhoz, majd ezzel implementáljuk a tanult iterációkat.

Alkalmazzunk is ezek közül egy olyat, amit értelmes az
$$
\left[\matrix{2 & -1 \cr -1 & 2}\right] x= \left[ \matrix{1 \cr 3} \right]
$$
egyenlet megoldására. Addig iteráljunk, míg két szomszédos iterált $\| \cdot \|_2$ szerinti távolsága $10^{-4}$ alá nem csökken.

In [ ]:
from dataclasses import dataclass
from functools import wraps
from typing import Callable

Matrix = np.ndarray
Vector = np.ndarray

@dataclass
class IterationResult:
    x: Vector
    success: bool
    abs_err: float
    rel_err: float
    step_num: int

@dataclass
class StoppingCondition:
    norm: Callable[(Vector,), float]
    atol: float
    rtol: float
    max_steps: int

def mn_iteration(
    extract_m: Callable[(Matrix,), Matrix],
    A: Matrix,
    b: Vector, 
    x0: Vector, 
    stopping_condition: StoppingCondition
) -> IterationResult:
    
    M = extract_m(A)
    N = M - A
    # M^{-1}N
    B = np.linalg.solve(M, N)
    # M^{-1}b
    r = np.linalg.solve(M, b)

    is_success = False
    x = x0
    for step_num in range(1, stopping_condition.max_steps+1):
        d = B@x + r - x;
        x = x + d
        
        # Ha d kicsi, azaz x_n es x_{n+1} kozel vannak, 
        # azaz x_n es f(x_n) kozel vannak,
        # azaz kozel vagyunk a fixponthoz.
        abs_err = stopping_condition.norm(d);

        if all([
            abs_err <= stopping_condition.atol,
            abs_err <= stopping_condition.rtol * stopping_condition.norm(x)
        ]):
            is_success = True
            break

    # tul sok lepes eseten sikertelen
    is_success &= (step_num <= stopping_condition.max_steps)

    return IterationResult(
        x,
        is_success,
        abs_err,
        abs_err/stopping_condition.norm(x),
        step_num
    )

In [ ]:
from functools import partial

def make_jor_iteration(omega):
    return partial(mn_iteration, lambda A: 1/omega*np.diag(np.diag(A)))

def make_sor_iteration(omega):
    return partial(mn_iteration, lambda A: 1/omega*np.diag(np.diag(A)) + np.tril(A, -1))

In [ ]:
jacobi_it = make_jor_iteration(omega=1)

In [ ]:
A = np.array([
    [2, -1],
    [-1, 2],
])

b = np.array([1, 3])
x0 = np.zeros((2,))

stopping_condition = StoppingCondition(
    partial(np.linalg.norm, ord=2), # nem kell a partial ha 2-es norma
    atol=1e-4,
    rtol=np.inf,
    max_steps=10**10,
)

In [ ]:
iteration_result = jacobi_it(A, b, x0, stopping_condition)

In [ ]:
iteration_result

In [ ]:
A @ iteration_result.x

### P3.* Feladat

Írjunk programot, amit adott $\lambda_1, \lambda_2$ valós sajátértékek és $q_1, q_2$ egymásra merőleges sajátvektorok esetén ábrázolja a megfelelő kvadratikus alakot, azaz a

$$
\mathbb{R}^2 \ni x \mapsto x^T A x
$$
függvényt, ahol $A = \lambda_1 q_1 q_1^T + \lambda_2 q_2 q_2^T$. Ábrázoljunk a sajátértékek előjele szerinti lehetséges esetekre 1-1 példát.

In [ ]:
import itertools

def make_quad_form(lam1: float, lam2: float, q1: np.ndarray, q2: np.ndarray):
    assert abs(q1 @ q2) < 1e-6
    # col. vectors
    q1 = q1.reshape((-1, 1))
    q2 = q2.reshape((-1, 1))

    Q = np.hstack((q1, q2))
    Q /= np.sqrt(sum(Q*Q))

    # (2, 2)
    A = Q @ np.diag([lam1, lam2]) @ Q.T

    # (n, m) -> (n, m) -> (n, m)
    def evaluate_quadratic_form_over_meshgrid_output(X,Y):
        # (2, n, m)
        T = np.stack((X, Y), 0)


        # Z[j, k] = SUM_(a, b) { T[a, j, k] * A[a, b] * T[b, j, k] }
        Z = np.einsum("ajk, ab, bjk -> jk", T, A, T)
        
        # (n, m)        
        return Z

    return evaluate_quadratic_form_over_meshgrid_output

# igazából elég megadni 1 (nem null)vektort, arra merőleges vektort tudnuk csinálni
# sajátértékek helyett is elég csak az egyik hosszát állítani tudni
def qplot_compare(v: np.ndarray = np.array([0, 1.]), magnitude: float = 1., resolution: int = 201):
    assert resolution >= 11
    assert int(resolution) == resolution
    assert v@v >= 1e-2
    assert magnitude > 0

    q1 = v
    q2 = np.array([[0, 1], [-1, 0]]) @ v
    
    xs = np.linspace(-1, 1, resolution)
    X, Y = np.meshgrid(xs, xs)

    fig, axes = plt.subplots(
        3, 3, 
        subplot_kw={"projection": "3d"}, 
        figsize=(10,10),
        sharex=True,
        sharey=True,
    )
    fig.suptitle('Kvadratikus alak grafikonja a 2 sajátérték előjele szerint')
    fig.supylabel(r'$\lambda_2$ előjele: -1, 0, 1')
    fig.supxlabel(r'$\lambda_1$ előjele: -1, 0, 1')

    # go through possible signs
    for s1, s2 in itertools.product([-1, 0, 1], [-1, 0, 1]):
        q = make_quad_form(s1*magnitude, s2, q1, q2)

        axis = axes[1-s1, s2+1]
        axis.plot_surface(X, Y, q(X, Y), cmap="jet")

        axis.axis('off')


In [ ]:
qplot_compare()

In [ ]:
## a 0 körül kevésbé para bola, kicsit más főiránnyal
qplot_compare(v=np.array([100, 100.]), magnitude=0.5)

### P4. Feladat

Implementáljuk a gradiens-módszert az optimális lépéshosszválasztással az SZPD-baloldalú lineáris egyenletrendszer iteratív megoldására.

Alkalmazzuk is ezt a módszert az 
$$
\left[\matrix{2 & -1 \cr -1 & 2}\right] x= \left[ \matrix{1 \cr 3} \right]
$$
egyenlet megoldására. Addig iteráljunk, míg két szomszédos iterált $\| \cdot \|_\infty$ szerinti távolsága $10^{-6}$ alá nem csökken.

In [ ]:
def grad_descent(
    A: Matrix,
    b: Vector, 
    x0: Vector, 
    stopping_condition: StoppingCondition
) -> IterationResult:

    is_success = False
    x = x0
    for step_num in range(1, stopping_condition.max_steps+1):
        # gradiens
        r = A@x - b
        # lepeshossz
        omega = r@r / (1e-16 + r@(A@r))

        d = - omega*r
        x = x + d
        
        # Ha a gradiens kicsi, akkor kozel vagyunk a fgv. szelsoertekhelyehez
        abs_err = stopping_condition.norm(d);
        if all([
            abs_err <= stopping_condition.atol,
            abs_err <= stopping_condition.rtol * stopping_condition.norm(x)
        ]):
            is_success = True
            break

    # tul sok lepes eseten sikertelen
    is_success &= (step_num <= stopping_condition.max_steps)

    return IterationResult(
        x,
        is_success,
        abs_err,
        abs_err/stopping_condition.norm(x),
        step_num
    )

In [ ]:
A = np.array([
    [2, -1],
    [-1, 2],
])
b = np.array([1,3])
x0 = np.zeros((2,))
stopping_condition = StoppingCondition(
    partial(np.linalg.norm, ord=np.inf),
    atol=1e-6,
    rtol=np.inf,
    max_steps=10**10,
)

res = grad_descent(A, b, x0, stopping_condition)

In [ ]:
res

In [ ]:
A@res.x